[Preliminary Analysis]
1. Access Post-processed interpolated daily fire perimeters 
2. Intersect each fire with ADS_10_19: Merged ADS Tree Mortality dataset 
3. Calculate % area of tree mortality per area burned 
    <% tree mortality = Area (ha) of TM / Total Area Burned]>
4. Compare with area burned per day 
5. Compare with rate of spread per day 

[Prediction]
1. Buffer day in advance see how much % mortality from the fire-line 
2. Use into model input 

In [ ]:
import arcpy
from arcpy import env
from arcpy.sa import *
arcpy.overwriteoutput = True

env.workspace = "F:\DriversFireProject\TEMP"

In [ ]:
import shutil
import pandas as pd
from datetime import datetime, date, time, timedelta
import requests
import zipfile
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
# get list of files based on directory and extension inputs 
def shpFiles(rootPath, ext):
    emptyList = []
    root = rootPath
    for path, subdirs, files in os.walk(root):
        for names in files: 
            if names.endswith(ext) and not names.startswith("._"):
                emptyList.append(path + '\\' + names)
    return(emptyList)

# Create new folder in root path 
def createFolder(rootPath, folderName): 
    folderPath = os.path.join(rootPath, folderName) 
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
    return folderPath + "\\" 

In [ ]:
# calculate total area burned (ha) for simplified interpolated surface 
rootPath = r'F:\DriversFireProject\NaturalNeighborResults\Daily'
simp = os.path.join(rootPath, 'SimplifiedSHP')
interFiles = shpFiles(simp, '.shp')
mergedADS = r'F:\TreeMortality\ADS_Dataset\Merged\Merged_ADS_10_19.shp'

for inter in interFiles: 
    # set up new lists per fire 
    fire = []
    year = [] 
    jd = [] 
    rptYr = [] 
    totalBurned = [] 
    tmArea = [] 
    # get fireName, year from path name 
    nm = inter.split("\\")
    name = nm[-1]
    nmm = name.split("_")
    fr = nmm[0]
    yr = nmm[1]
    print(yr, fr)
    
    # Calculate TOTAL burned area (ha) 
    arcpy.DeleteField_management(inter, "BurnedArea")
    arcpy.AddField_management(inter, "BurnedArea", "DOUBLE")
    arcpy.CalculateField_management(inter, "BurnedArea", "!shape.area@HECTARES!", "PYTHON", "")
    
    # Set up new folders 
    outpath = r'F:\TreeMortality\DailyFireIntersects'
    yrFolder = createFolder(outpath, str(yr))
    frFolder = createFolder(yrFolder, fr)
    
    # Intersect merged ADS with each fire perimeters 
    intersect = arcpy.Intersect_analysis([inter, mergedADS], frFolder + name)
    
    # Calculate area of Tree Mortality per day 
    arcpy.AddField_management(intersect, "TMArea", "DOUBLE")
    arcpy.CalculateField_management(intersect, "TMArea", "!shape.area@HECTARES!", "PYTHON", "")
    with arcpy.da.SearchCursor(intersect, ['BurnedArea', 'TMArea', 'gridcode', 'RPT_YR']) as cursor:
        for row in cursor:
            fire.append(fr)
            year.append(yr)
            rptYr.append(row[3])
            totalBurned.append(row[0])
            jd.append(row[2])
            tmArea.append(row[1])
    
    # Create DF
    multiDirDF = pd.DataFrame({'Fire': fire, 'FR_Year': year,'FR_Day':jd, 
                               'TM_Year': rptYr, 'Burned_Area(ha)': totalBurned, 'TM_Area(ha)': tmArea})
    
    # Groupby Fire, Fr_year, Fr_Day, TM_Year, Burned_Area(ha) >> aka ADD up TM_AREA per RPTYEAR 
    grouped = multiDirDF.groupby(['Fire', 'FR_Year', 'FR_Day', 'TM_Year', 'Burned_Area(ha)'])['TM_Area(ha)'].agg('sum').reset_index()
    # Calculate ratio TM_AREA(ha) / Burned_AREA(ha) 
    grouped['TM_Ratio'] = grouped['TM_Area(ha)'] / grouped['Burned_Area(ha)']
    
    CSVPath = r'F:\TreeMortality\CSV_TMPerFire'
    yrCSVFolder = createFolder(CSVPath, str(yr))
    grouped.to_csv(yrCSVFolder + fr + "_" + str(yr) + '_TM.csv') 
    
    # pivot Table 
    grouped['TM_Year'] = grouped['TM_Year'].astype(str)
    PIVtable = pd.pivot_table(grouped, values=['TM_Area(ha)', 'TM_Ratio'], index=['Fire', 'FR_Year', 'FR_Day', 'Burned_Area(ha)'],
                        columns=['TM_Year'], aggfunc=np.sum).reset_index()

    PIVtable.columns = [' '.join(col).strip() for col in PIVtable.columns.values]

    CSVPath2 = r'F:\TreeMortality\CSV_TMPerFire_PIV'
    yrCSVFolder2 = createFolder(CSVPath2, str(yr))
    
    PIVtable.to_csv(yrCSVFolder2 + fr + "_" + str(yr) + '_TMPIV.csv')         
        
            


In [ ]:
multiDirDF
# Groupby Fire, Fr_year, Fr_Day, TM_Year, Burned_Area(ha) >> aka ADD up TM_AREA per RPTYEAR 
grouped = multiDirDF.groupby(['Fire', 'FR_Year', 'FR_Day', 'TM_Year', 'Burned_Area(ha)'])['TM_Area(ha)'].agg('sum').reset_index()
# Calculate ratio TM_AREA(ha) / Burned_AREA(ha) 
grouped['TM_Ratio'] = grouped['TM_Area(ha)'] / grouped['Burned_Area(ha)']
grouped['TM_Year'] = grouped['TM_Year'].astype(str)
grouped

SPLIT csv into BEFORE and AFTER Fire: 
1. [Before]: relationship between how tree mortality affects fires behavior 
2. [After]: relationship between how fire can induce tree mortality (stress) 

In [ ]:
# Merge sll piv csv files 
CSVPath = r'F:\TreeMortality\CSV_TMPerFire'
csvFiles = shpFiles(CSVPath, '.csv')
outPath = r'F:\TreeMortality\\'
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in csvFiles])
#export to csv
combined_csv.to_csv( outPath + "MergedALL_TM.csv", index=False, encoding='utf-8-sig')

In [ ]:
combined_csv['TM_Year'] = combined_csv['TM_Year'].astype(str)
PIVtable = pd.pivot_table(combined_csv, values=['TM_Ratio'], index=['Fire', 'FR_Year', 'FR_Day', 'Burned_Area(ha)'],
                    columns=['TM_Year'], aggfunc=np.sum).reset_index()

PIVtable = PIVtable.fillna(0)
PIVtable = PIVtable.round(5)
PIVtable.columns = [' '.join(col).strip() for col in PIVtable.columns.values]
PIVtable.to_csv( outPath + "MergedALL_TM_PIV.csv", index=False, encoding='utf-8-sig')

In [ ]:
# 1. [Before]: relationship between how tree mortality affects fires behavior 
# Keep rows only where TM_Area IF YM_YEAR is less than equal to FR_Year 
combined_csv['FR_Year'] = combined_csv['FR_Year'].astype('int64')
combined_csv['TM_Year'] = combined_csv['TM_Year'].astype('int64')
combined_csv['Group'] = np.where((combined_csv['FR_Year'] > combined_csv['TM_Year']), 'BeforeFire', 'AfterFire')

In [ ]:
TM_BeforeFire = pd.read_csv(outPath + "BeforeFire_TM.csv", index_col=0)

In [ ]:
# look at TM YEARS -1 , -3, -all) 
TM_BeforeFire['YearPrior'] = TM_BeforeFire['FR_Year'] - TM_BeforeFire['TM_Year']
TM_BeforeFire

In [ ]:
# one year prior 
onyear = TM_BeforeFire[TM_BeforeFire['YearPrior'] < 1]
onyearGroup = onyear.groupby(['Fire', 'FR_Year', 'FR_Day', 'Burned_Area(ha)'])['TM_Area(ha)'].agg('sum').reset_index()
onyearGroup['TM_Ratio'] = onyearGroup['TM_Area(ha)']/ onyearGroup['Burned_Area(ha)']
onyearGroup

In [ ]:
# three years prior
threeYear = TM_BeforeFire[TM_BeforeFire['YearPrior'] < 3]
threeYear = threeYear.groupby(['Fire', 'FR_Year', 'FR_Day', 'Burned_Area(ha)', 'YearPrior'])['TM_Area(ha)'].agg('sum').reset_index()
threeYear['TM_Ratio'] = threeYear['TM_Area(ha)']/ threeYear['Burned_Area(ha)']
threeYear['log_Area'] = np.log(threeYear['Burned_Area(ha)'])

In [ ]:
g = sns.FacetGrid(threeYear, col="FR_Year", hue="YearPrior")
g = (g.map(plt.scatter, "log_Area", "TM_Ratio", edgecolor="w")
        .add_legend())

In [ ]:
# all years prior 
AllYear = TM_BeforeFire.groupby(['Fire', 'FR_Year', 'FR_Day', 'Burned_Area(ha)'])['TM_Area(ha)'].agg('sum').reset_index()
AllYear['TM_Ratio'] = AllYear['TM_Area(ha)']/ AllYear['Burned_Area(ha)']
AllYear['log_Area'] = np.log(AllYear['Burned_Area(ha)'])

g = sns.FacetGrid(AllYear, col="FR_Year")
g = (g.map(plt.scatter, "log_Area", "TM_Ratio", edgecolor="w")
        .add_legend())

In [ ]:
AllYear = TM_BeforeFire.groupby(['Fire', 'FR_Year'])['TM_Area(ha)', 'Burned_Area(ha)'].agg('sum').reset_index()
AllYear['Ratio'] = AllYear['TM_Area(ha)']/ AllYear['Burned_Area(ha)']
AllYear.sort_values(by=['TM_Area(ha)'], ascending=False)

In [ ]:
# 2. [After]: relationship between how fire can induce tree mortality (stress) 
TM_AfterFire = pd.read_csv(outPath + "AfterFire_TM.csv", index_col=0)